In [2]:
!pip install selenium

  Using cached https://files.pythonhosted.org/packages/80/d6/4294f0b4bce4de0abf13e17190289f9d0613b0a44e5dd6a7f5ca98459853/selenium-3.141.0-py2.py3-none-any.whl


distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
import pandas as pd
import bs4
import selenium
import requests
from selenium import webdriver

In [98]:
OnePlus_6T = "http://tiny.cc/sck34y"
page = requests.get(OnePlus_6T).text
page

'<html>\r\n<head><title>403 Forbidden</title></head>\r\n<body bgcolor="white">\r\n<center><h1>403 Forbidden</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n'

In [99]:
browser = webdriver.Chrome('chromedriver.exe')

In [100]:
OnePlus_6T = "http://tiny.cc/sck34y"
browser.get(OnePlus_6T)

In [101]:
products = browser.find_elements_by_css_selector('div#dp')
len(products)

1

In [178]:
def get_info(oneplus, selector, attribute):
    try:
        tag = oneplus.find_element_by_css_selector(selector)
        tag = tag.get_attribute(attribute)
    except:
        tag = None
    return tag

df_products = pd.DataFrame(columns = ['Title','Description','Image',
                                      'price_before_exchange','price_after_exchange', 
                                      'Product_Colours','No. of Reviews','Star_rating',
                                      'Technical_Details','Reviews'])
for oneplus in products:
    Title = get_info(oneplus, 'span.a-size-large','innerHTML')
    
    # get product description 
    Description_parent = oneplus.find_element_by_css_selector('div#productDescription')
    Description = get_info(Description_parent,'div','innerHTML')  
    
    Image = get_info(oneplus,'div#imgTagWrapperId','innerHTML')
    
    price_before_exchange = get_info(oneplus, 'span.priceBlockStrikePriceString.a-text-strike','innerHTML')
    
    price_after_exchange = get_info(oneplus, 'span#priceblock_dealprice','innerHTML')
    
    Product_Colours = get_info(oneplus, 'span.selection','innerHTML')
    #Product_Colours =oneplus.find_element_by_css_selector('span.selection').text 
    
    No_of_Reviews = oneplus.find_element_by_css_selector('h2[data-hook="total-review-count"]').text 
    
    Star_rating = get_info(oneplus, 'span.a-icon-alt','innerHTML')
    
    Technical_Details = oneplus.find_element_by_css_selector('div.attrG').text
    
    #Most_Recent_100_Reviews = get_info(oneplus, 'span','innerHTML')
    #Most_Recent_100_Reviews = oneplus.find_element_by_css_selector('div.data-hook').text
    
    reviews_parent = oneplus.find_element_by_css_selector('div.a-expander-content.reviewText.review-text-content.a-expander-partial-collapse-content')
    reviews_child = reviews_parent.find_element_by_css_selector('span')
    try:
        reviews = reviews_child.find_element_by_css_selector('span')
        reviews = reviews.get_attribute('innerHTML')
    except:
        reviews = reviews_child.get_attribute('innerHTML')
    
    curr_row = {
        'Title': Title,
        'Description': Description,
        'Image': Image,
        'price_before_exchange': price_before_exchange,
        'price_after_exchange': price_after_exchange,
        'Product_Colours': Product_Colours,
        'No. of Reviews': No_of_Reviews,
        'Star_rating': Star_rating,
        'Technical_Details': Technical_Details,
        'Reviews': reviews
    }
    df_products = df_products.append(curr_row,ignore_index = True)
df_products

,Title,Description,Image,price_before_exchange,price_after_exchange,Product_Colours,No. of Reviews,Star_rating,Technical_Details,Reviews
0,\n \n \n ...,\n \n \n \n ...,"\n\t <img alt=""OnePlus 6T (Midnight...","<span class=""currencyINR"">&nbsp;&nbsp;</span>...","<span class=""currencyINR"">&nbsp;&nbsp;</span> ...",8+128 GB,"15,564 customer reviews",4.5 out of 5 stars,OS Android\nRAM 8 GB\nItem Weight 186 g\nProdu...,I got this phone on Friday evening.<br><br>Pro...


In [173]:
df_products['Title'] = df_products['Title'].str.replace('[\n ]', '')
df_products['Description'] = df_products['Description'].str.replace('[\n ]', '')
df_products['Image'] = df_products['Image'].str.replace('[\n\t ]', '')
df_products['Technical_Details'] = df_products['Technical_Details'].str.replace('[\n]', '')
df_products['Reviews'] = df_products['Reviews'].str.replace('[<br>]', '')

In [174]:
df_products['price_after_exchange'] = df_products['price_after_exchange'].str.replace('[^0-9 ]', '')
df_products['price_before_exchange'] = df_products['price_before_exchange'].str.replace('[^0-9 ]', '')

In [175]:
df_products

,Title,Description,Image,price_before_exchange,price_after_exchange,Product_Colours,No. of Reviews,Star_rating,Technical_Details,Reviews
0,"OnePlus6T(MidnightBlack,8GBRAM,128GBStorage)",Sizename:<strong>8+128GB</strong>&nbsp;|&nbsp;...,"<imgalt=""OnePlus6T(MidnightBlack,8GBRAM,128GBS...",4199900,3299900,8+128 GB,"15,564 customer reviews",4.5 out of 5 stars,OS AndroidRAM 8 GBItem Weight 186 gProduct Dim...,I got this phone on Fiday evening.Pos:Geat att...


## Sentiment Analysis

In [179]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

C:\Users\Ananya\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [180]:
sentiment = SentimentIntensityAnalyzer()